In [1]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.nn.functional as F
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import os
import cv2
import pandas as pd
from skimage import io
from torch.optim.lr_scheduler import StepLR

# import scripts
from scripts.saveResults import  *
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

Dataset 

In [2]:

class SolarRadiance(Dataset):
    def __init__(self, root_dir, labels, transform):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        # self.data = self.load_dataset()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img_path = self.labels.iloc[index,0]
        target = self.labels.iloc[index,1]

        image = io.imread(img_path)
        image = cv2.imread(img_path)#, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (60, 80))

        if self.transform:
             image = self.transform(image)

        y_label = torch.tensor(target)

        return image, y_label

In [3]:
def load_dataset(root_dir):
        ds = pd.DataFrame()
        dates = os.listdir(root_dir)

        try:
            for date in dates:
                infrared_folder = os.path.join(root_dir, date, "infrared")
                pyranometer_folder = os.path.join(root_dir, date, "pyranometer")
                csv_path = os.path.join(pyranometer_folder, "{date}.csv".format(date=date))
                if not os.path.exists(csv_path):
                    print("Skipping date {date} because it does not have both infrared and pyranometer folders".format(date=date))
                    continue
                ds_temp = getDs(infrared_folder, csv_path)
                
                # append the dataframe in the final dataframe
                # ds = ds.append(ds_temp)
                ds = pd.concat([ds, ds_temp], ignore_index=True)

                ds['name'] = ds['name'].apply(lambda img: os.path.join(root_dir, date, 'infrared', img))
        except Exception as e:
            print(e)
        return ds

def getDs(path, labels):
    pyranometer = pd.read_csv(labels)
    images = os.listdir(path)
    
   
    #convert column 1 to int
    X = pyranometer.iloc[:,0].astype(int)

    #convert to image names
    pyranometer.iloc[:,0] = X.apply(lambda x: str(x) + 'IR.png')
    
    # Filter pyranometer DataFrame based on the 'x' column

    filtered_pyranometer = pyranometer[pyranometer.iloc[:,0].isin(images)]
    # Display the result
    filtered_pyranometer.columns = ['name', 'value']

    filtered_pyranometer = filtered_pyranometer.drop_duplicates(subset='name')
    return filtered_pyranometer

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load Data

In [5]:

# Hyperparameters
in_channel = 1
batch_size = 50
num_epochs = 50
loss = 1 # if loss = 0 the model will be trained with RMSE loss and vice versa
lr=0.01 # learning rate
random_seed = 42


In [6]:
# create result directory
result_dir = 'result/kd'

In [7]:
train_dir = 'F:/Girasol/extracted/train/'
train_data  = load_dataset(train_dir)
torch.manual_seed(random_seed)
train_set = SolarRadiance(root_dir= train_dir, labels=train_data, transform = transforms.Compose([transforms.ToTensor()]))
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

val_dir = 'F:/Girasol/extracted/val/'
val_data  = load_dataset(val_dir)
torch.manual_seed(random_seed)
random_seed = 42
val_set = SolarRadiance(root_dir= val_dir, labels=val_data, transform = transforms.Compose([transforms.ToTensor()]))
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True)

Model

In [8]:
import torch
import torch.nn as nn

class CNNRegression(nn.Module):
    def __init__(self, image_size, num_channels):
        super(CNNRegression, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(int(60/4) * int(80/4) * 32, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [9]:
student = CNNRegression(60, 3).to(device)

# Load the pre-trained model
# student.load_state_dict(torch.load('results/batch_50_lr_0.01_loss_1/2024-01-01_00-29-03/best_model.pth'))
student.to(device)
student.train()

CNNRegression(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9600, out_features=64, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

Loss and Optimizers

In [10]:
if loss == 0:
    # create a function (this my favorite choice)
    def RMSELoss(yhat,y):
        return torch.sqrt(torch.mean((yhat-y)**2))
    criterion = RMSELoss
else:
    # Define the model, loss function, and optimizer
    criterion = nn.MSELoss()
    

optimizer = torch.optim.Adam(student.parameters(), lr=lr)
# Create a StepLR scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

# set initial lowest_loss to an infinite number
lowest_loss = 10000000


In [11]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained MobileNet model without its final classification layer
mobilenet = models.mobilenet_v2(pretrained=True)
# Remove the last layer (classification layer)
mobilenet = nn.Sequential(*list(mobilenet.children())[:-1])

# Add custom layers for regression
class LinearRegressionHead(nn.Module):
    def __init__(self, in_features):
        super(LinearRegressionHead, self).__init__()
        self.fc = nn.Linear(in_features, 1)  # Output 1 value for linear regression
    
    def forward(self, x):
        x = self.fc(x)
        return x

# Append the custom regression head to the pre-trained model
in_features = mobilenet[-1][-1].out_channels  # Get the number of output channels from the last layer of MobileNet

c:\Users\nijhu\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nijhu\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
losses = []

# Specify the path to the saved model checkpoint
teacher_checkpoint = 'result/teacher/teacher.pth'
teacher = nn.Sequential(
    mobilenet,
    nn.AdaptiveAvgPool2d(1),  
    nn.Flatten(),
    LinearRegressionHead(in_features)
)

# Load the saved model state_dict
teacher.load_state_dict(torch.load(teacher_checkpoint))
teacher.to(device)
teacher.eval()

# Set the distillation parameters (alpha, temperature)
alpha = 0.8  # Weight for hard targets
temperature = 10  # Sharpness of soft targets
# KD MSE WITH TEMP 10
from tqdm import tqdm
for epoch in range(num_epochs):
    scheduler.step()
    print('Epoch ',epoch)
    for i, (inputs, targets) in enumerate(tqdm(train_loader)):
        # Zero the gradients
        optimizer.zero_grad()
        inputs = inputs.to(device)
        targets = targets.to(device)

        ########### Distillation Loss ###########
        # Forward pass
        outputs_student = student(inputs)
        with torch.no_grad():
            outputs_teacher = teacher(inputs)

        # Calculate classification loss
        hard_loss = criterion(outputs_student[:, 0], targets.float())

        soft_student_outputs = F.sigmoid(outputs_student/temperature)   
        soft_teacher_outputs = F.sigmoid(outputs_teacher/temperature)
        distillation_loss = criterion(outputs_student, outputs_teacher)
        # Total loss
        loss = alpha * hard_loss + (1 - alpha) * distillation_loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print('hard_loss: ', hard_loss)
    print('distillation loss: ', distillation_loss)
    print('calculated loss', loss)
    # Evaluate the model on the test data
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = student(inputs)
            loss = criterion(outputs[:,0], targets.float())
            total_loss += loss.item()
        mean_loss = total_loss / len(val_loader)
        print(f'val Loss: {mean_loss:.4f}')
        losses.append(mean_loss)
        if  mean_loss<lowest_loss:
            print('Mean_loss '+str(mean_loss)+'lowest_loss'+str(lowest_loss))
            print('RMSE LOSS: '+str(torch.sqrt(torch.tensor(mean_loss))))
            lowest_loss = mean_loss
            torch.save(student.state_dict(), os.path.join(result_dir,'kd_model.pth'))    

c:\Users\nijhu\anaconda3\envs\pytorch\Lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch  0


100%|██████████| 931/931 [05:48<00:00,  2.67it/s]


hard_loss:  tensor(2362.7209, device='cuda:0', grad_fn=<MseLossBackward0>)
distillation loss:  tensor(1977.1278, device='cuda:0', grad_fn=<MseLossBackward0>)
calculated loss tensor(2285.6023, device='cuda:0', grad_fn=<AddBackward0>)
val Loss: 5125.0253
Mean_loss 5125.025346427128lowest_loss10000000
RMSE LOSS: tensor(71.5893)
Epoch  1


100%|██████████| 931/931 [01:49<00:00,  8.46it/s]


...

hard_loss:  tensor(6411.7554, device='cuda:0', grad_fn=<MseLossBackward0>)
distillation loss:  tensor(5260.9507, device='cuda:0', grad_fn=<MseLossBackward0>)
calculated loss tensor(6181.5947, device='cuda:0', grad_fn=<AddBackward0>)
val Loss: 2542.7772
Epoch  48


100%|██████████| 931/931 [02:42<00:00,  5.73it/s]


hard_loss:  tensor(2297.8765, device='cuda:0', grad_fn=<MseLossBackward0>)
distillation loss:  tensor(2276.0295, device='cuda:0', grad_fn=<MseLossBackward0>)
calculated loss tensor(2293.5071, device='cuda:0', grad_fn=<AddBackward0>)
val Loss: 2544.2747
Epoch  49


100%|██████████| 931/931 [02:42<00:00,  5.73it/s]


hard_loss:  tensor(3065.8794, device='cuda:0', grad_fn=<MseLossBackward0>)
distillation loss:  tensor(1977.1700, device='cuda:0', grad_fn=<MseLossBackward0>)
calculated loss tensor(2848.1377, device='cuda:0', grad_fn=<AddBackward0>)


Results

In [ ]:
# save the losses per epoch
import pickle

# Save the list to a file
with open(result_dir+'/losses.pkl', 'wb') as file:
    pickle.dump(losses, file)

In [13]:
import pickle

# Load the list from the file
losses_path = os.path.join(result_dir,'losses.pkl')
with open(losses_path, 'rb') as file:
    losses = pickle.load(file)
print(losses)

[5125.025346427128, 3783.5929223422345, 3180.622602791622, 2934.1552860654633, 2682.950215306775, 2804.6780453385977, 2570.101334144329, 2580.789471330314, 2553.529848822232, 2544.90777219575, 2542.0412502946524, 2551.683973641231, 2542.207899422481, 2548.5630119586813, 2540.95843979408, 2544.0871192669047, 2542.98259077401, 2540.678268958782, 2546.91947042531, 2545.6182114173625, 2540.4398382778827, 2546.107275074926, 2544.701911137022, 2548.604067046067, 2542.0297683189656, 2543.206382488382, 2541.7154630463697, 2544.921569034971, 2543.365880505792, 2544.661238045528, 2544.6026700776197, 2544.0797355914938, 2546.034806350182, 2544.4788402688914, 2542.1497013486664, 2542.3265354551118, 2540.482212987439, 2544.016759412042, 2544.032019253435, 2540.46507000101, 2543.471394505994, 2542.1931073418978, 2545.98276072535, 2543.6802678601493, 2542.030156628839, 2541.7148569041283, 2541.5460294526197, 2542.7771501212283, 2544.274728709254, 2541.300435559503]


In [ ]:
# import os
# os.system("shutdown /s /t 1")

In [ ]:
saveLossDiagram(num_epochs,losses,result_dir)

In [14]:
# Specify the path to the saved model checkpoint
model_checkpoint_path = 'result\kd\kd_model.pth'
# Load the saved model state_dict
student.load_state_dict(torch.load(model_checkpoint_path))

<All keys matched successfully>

Load model and get results

validation

In [15]:
outputss = []
targetss = []

# choose loss to evaluate with 
if loss == 0:
    criterion = nn.MSELoss()
else:
    def RMSELoss(yhat,y):
        return torch.sqrt(torch.mean((yhat-y)**2))
    criterion = RMSELoss

# Evaluate the model on the validation data
with torch.no_grad():
    total_loss = 0
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = student(inputs)
        outputss.append(outputs[:,0].tolist())
        targetss.append(targets.tolist())
        loss = criterion(outputs[:,0], targets.float())
        total_loss += loss.item()
    mean_loss = total_loss / len(val_loader)
    if loss == 0:
        print("Batch Size %d , Epochs %d , Learning Rate %f , Loss %f"%(batch_size, num_epochs, lr, lowest_loss))
        print(f'MSE Loss: {mean_loss:.4f}')
        SaveResults(result_dir, batch_size, num_epochs, lr, mean_loss, lowest_loss)
    else:
        print("Batch Size %d , Epochs %d , Learning Rate %f , Loss %f"%(batch_size, num_epochs, lr, lowest_loss))
        print(f'MSE Loss: {mean_loss:.4f}')
        SaveResults(result_dir, batch_size, num_epochs, lr,lowest_loss, mean_loss)

Batch Size 50 , Epochs 50 , Learning Rate 0.010000 , Loss 10000000.000000
MSE Loss: 49.9668
result/kd\results.csv


Flatten the data

In [ ]:
outputs = []
for i in outputss:
    for j in i:
        outputs.append(j)

targets = []
for i in targetss:
    for j in i:
        targets.append(j)

In [ ]:
observedvsPredicted(result_dir, targets, outputs)

Train and Shut Down

In [ ]:
# import os
# os.system("shutdown /s /t 1")